In [4]:
%matplotlib inline
import pandas as pd

from datetime import datetime
from math import *
import numpy as np
import sys
import numpy as np
import warnings

import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
from matplotlib import font_manager, rc
font_location = 'C:/Windows/Fonts/gulim.ttc'
font_name = font_manager.FontProperties(fname = font_location).get_name()
rc('font', family=font_name)


import dtw

import operator

In [13]:
card = pd.read_csv('card_20200717.csv')
covid=pd.read_csv('Time.csv')

In [14]:
card.columns = ['날짜', '행정동 코드', '행정동', '업종 코드', '업종', '판매건수', '판매금액']
card['날짜'] = card['날짜'].astype('str').apply(lambda x: x[:4]+'-'+x[4:6]+'-'+x[6:])
card['날짜']=pd.to_datetime(card['날짜'], format="%Y-%m-%d")
card['행정동 코드'] = card['행정동 코드'].astype('str').apply(lambda x: x[:-2]).astype('int')
card['업종 코드'] = card['업종 코드'].astype(str)
card['업종 중분류'] = card['업종 코드'].str.slice(start=0, stop=2)


covid.columns = ['날짜', '시간대', '검사', '음성', '누적확진', '격리해제','사망']
covid['날짜']=pd.to_datetime(covid['날짜'], format="%Y-%m-%d")
covid['실확진자']=covid['누적확진'].diff().apply(pd.to_numeric)

mask1 = (card['날짜'] >= '2020-01-20')
mask3= (covid['날짜'] <= '2020-06-14')

card=card.loc[mask1]
covid=covid.loc[mask3]

In [15]:
date = list(card['날짜'].unique())
category = list(card['업종'].unique())
item = [date, category]

from itertools import product
card_temp = pd.DataFrame(product(*item), columns= ['날짜','업종'])

card = pd.merge(card,card_temp,on=['날짜','업종'], how='right')
card = card.fillna(0)
card = card.groupby(['날짜','업종']).mean().reset_index() # 날짜별로 업종이 가나다 순서대로 정렬되는 효과

In [16]:
card_compare = card.copy()
import datetime as dt

card['주'] = card['날짜'].dt.week #주 변수 추가
card = card.groupby(['업종','주']).mean().reset_index()
card['판매건수'].fillna(0)
card['판매금액'].fillna(0)

In [17]:
card

,날짜,업종,행정동 코드,판매건수,판매금액,주,월
0,2020-01-04,1급 호텔,1.114055e+07,38.000000,6.941145e+06,1,1
1,2020-01-04,2급 호텔,0.000000e+00,0.000000,0.000000e+00,1,1
2,2020-01-04,CATV,0.000000e+00,0.000000,0.000000e+00,1,1
3,2020-01-04,CATV홈쇼핑,1.154568e+07,2384.500000,2.736035e+08,1,1
4,2020-01-04,L P G,0.000000e+00,0.000000,0.000000e+00,1,1
...,...,...,...,...,...,...,...
40295,2020-06-14,화 랑,1.132301e+07,5.080000,3.173280e+05,24,6
40296,2020-06-14,화 원,1.144383e+07,9.407514,3.033937e+05,24,6
40297,2020-06-14,화 장 품,1.143711e+07,34.792916,1.602415e+06,24,6
40298,2020-06-14,화물 운송,1.141588e+07,71.948454,7.139541e+05,24,6


In [9]:
card

,업종,주,행정동 코드,판매건수,판매금액,월
0,1급 호텔,1,5.570275e+06,19.000000,3.470572e+06,1.000000
1,1급 호텔,2,6.366029e+06,23.000000,3.767738e+06,1.000000
2,1급 호텔,3,9.549043e+06,28.000000,3.316541e+06,1.000000
3,1급 호텔,4,8.148415e+06,17.711262,3.480931e+06,1.000000
4,1급 호텔,5,1.139836e+07,27.726274,4.324041e+06,1.285714
...,...,...,...,...,...,...
6235,화방표구점,20,1.140869e+07,11.921409,7.188958e+05,5.000000
6236,화방표구점,21,1.139273e+07,13.174195,7.079540e+05,5.000000
6237,화방표구점,22,1.140849e+07,12.824208,7.980002e+05,5.000000
6238,화방표구점,23,1.138337e+07,10.998074,7.476036e+05,6.000000


In [ ]:
fpopl_compare=fpopl.copy()

fpopl['주']=fpopl['날짜'].dt.week #주 변수 추가
fpopl = fpopl.groupby(['주']).mean().reset_index()

In [ ]:
covid_compare=covid.copy()

covid['주']=covid['날짜'].dt.week #주 변수 추가
covid = covid.groupby(['주']).sum().reset_index()

In [ ]:
card['판매건수변환'] = card.groupby('업종')['판매건수'].transform(lambda x: (x-x.min())/(x.max() - x.min()))
card['판매건수변환2'] = card.groupby('업종')['판매건수'].transform(lambda x: 1-((x-x.min())/(x.max() - x.min())))

In [ ]:
covid['확진자수변환']= covid['실확진자'].transform(lambda x: (x-x.min())/(x.max() - x.min()))

In [ ]:
kind = list(card['업종'].unique())

In [ ]:
distance={}
for i in kind:
    try:         
        a=dtw.dtw(covid['확진자수변환'],card.loc[(card['업종']==i)]['판매건수변환']).distance
        distance[i]=a
        print(i,a)
    except:
        print('++++++++++++++\n',i,'error\n','+++++++++++++++++++++')

In [ ]:
distance

In [ ]:
max(distance.items(), key=operator.itemgetter(1))

In [ ]:
min(distance.items(), key=operator.itemgetter(1))

In [ ]:
distance2={}
for i in kind:
    try:         
        a=dtw.dtw(covid['확진자수변환'],card.loc[(card['업종']==i)]['판매건수변환2']).distance

        distance2[i]=a
        print(i,a)
    except:
        print('++++++++++++++\n',i,'error\n','+++++++++++++++++++++')

In [ ]:
max(distance2.items(), key=operator.itemgetter(1))

In [ ]:
min(distance2.items(), key=operator.itemgetter(1))

In [ ]:
card_temp = card.copy()
fig, ax = plt.subplots(1,1,figsize=(14,5))
for i in range(0, len(set(card_temp['업종']))):
    if i//10 == 0:
        ax.plot(np.array(card_temp[card_temp['업종'] == kind[i]]['판매건수변환']),marker='o', linewidth=2,label=kind[i])
ax.legend()
ax.set_title('산업추이')

In [ ]:
plt.plot(covid['확진자수변환'],marker='o', linewidth=2)
plt.title('확진자수변환')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(14,5))
ax.plot(np.array(covid['확진자수변환']),marker='o', linewidth=2,label='확진자')
ax.plot(np.array(card_temp[card_temp['업종'] == '화   원']['판매건수변환']),marker='o', linewidth=2,label='화원')
ax.plot(np.array(card_temp[card_temp['업종'] == '연 쇄 점']['판매건수변환']),marker='o', linewidth=2,label='연 쇄 점')

ax.legend()

In [ ]:
from scipy.spatial.distance import euclidean 

In [ ]:
euclidean_dis={}
for i in kind: 
    try:
        a=euclidean(covid['확진자수변환'],card.loc[(card['업종']==i)]['판매건수변환'])

        euclidean_dis[i]=a
        print(i,a)
    except:
        print('error\n',i,'error\n')

In [ ]:
max(euclidean_dis.items(), key=operator.itemgetter(1))

In [ ]:
min(euclidean_dis.items(), key=operator.itemgetter(1))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(14,5))
ax.plot(np.array(covid['확진자수변환']),marker='o', linewidth=2,label='확진자')
ax.plot(np.array(card_temp[card_temp['업종'] == '고속 버스']['판매건수변환']),marker='o', linewidth=2,label='고속 버스')
ax.plot(np.array(card_temp[card_temp['업종'] == '대학등록금']['판매건수변환']),marker='o', linewidth=2,label='대학등록금')

ax.legend()

In [ ]:
euclidean_dis2={}
for i in kind: 
    try:
        a=euclidean(covid['확진자수변환'],card.loc[(card['업종']==i)]['판매건수변환2'])

        euclidean_dis2[i]=a
        print(i,a)
    except:
        print('error\n',i,'error\n')

In [ ]:
max(euclidean_dis2.items(), key=operator.itemgetter(1))

In [ ]:
min(euclidean_dis2.items(), key=operator.itemgetter(1))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(14,5))
ax.plot(np.array(covid['확진자수변환']),marker='o', linewidth=2,label='확진자')
ax.plot(np.array(card_temp[card_temp['업종'] == '택시회사']['판매건수변환2']),marker='o', linewidth=2,label='택시회사')
ax.plot(np.array(card_temp[card_temp['업종'] == '이 용 원']['판매건수변환2']),marker='o', linewidth=2,label='이 용 원')

ax.legend()